In [1]:
from web3 import Web3
import pprint as pp

In [2]:
w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/c6e7f057427949da8d51837a25816997'))
w3.isConnected()

True

In [15]:
from Arbie.Contracts.contract import Address, ContractFactory, Network
from Arbie.Contracts.uniswap import Factory, Pair

# https://uniswap.info/pair/0xbb2b8038a1640196fbe3e38816f3e67cba72d940
# WBTC-ETH Pair
#pair_address = Address("0xBb2b8038a1640196FbE3e38816F3e67Cba72D940")


# https://uniswap.info/pair/0x94cdd18f53a8f3ec9a3ec0cbe897aed5ea009c43
# DAI-yDAI Pair
#pair_address = Address("0x94cDd18F53a8f3EC9A3Ec0CBE897aED5ea009c43")

uni_factory = ContractFactory(w3, Factory).load_contract(
    Address(""), network=Network.mainnet)

pair_address = uni_factory.get_pair_address(3)

In [16]:
pair = ContractFactory(w3, Pair).load_contract(Address(""), address=pair_address)

tokens = pair.get_tokens()
pp.pprint(tokens)

amm = pair.create_amm()
amm.spot_price(tokens[0].create_token(), tokens[1].create_token())

[GenericToken, name: Dai Stablecoin, address: 0x6B175474E89094C44Da98b954EedeAC495271d0F,
 GenericToken, name: Wrapped Ether, address: 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2]


350.7147669626478